# Import Packages and Data

In [3]:
!pip install sklearn

     |████████████████████████████████| 6.7MB 3.2MB/s eta 0:00:01     |███                             | 604kB 3.2MB/s eta 0:00:02     |████████▊                       | 1.8MB 3.2MB/s eta 0:00:02     |██████████████████              | 3.7MB 3.2MB/s eta 0:00:01     |█████████████████████▍          | 4.5MB 3.2MB/s eta 0:00:01     |███████████████████████▎        | 4.9MB 3.2MB/s eta 0:00:01     |█████████████████████████▋      | 5.3MB 3.2MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [4]:
import tensorflow
import keras
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import pickle
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import RMSprop, Adadelta, Adam

In [5]:
data = pd.read_csv('../data/buffer_overflow_data.csv.gz')
labels = data.copy()
del labels['Unnamed: 0']
del labels['Unnamed: 0.1']
del labels['filename']
del labels['code']
del labels['flaw']
del labels['flaw_loc']
labels = labels.drop_duplicates().sort_values('testcase_ID').reset_index(drop=True)

In [6]:
x = pd.read_csv("../data/buffer_overflow_graph_embeddings.csv.gz")
x['testcase_ID'] = x['type']
del x['type']
x = x.sort_values("testcase_ID").reset_index(drop=True)

In [7]:
y = labels['bug']
x = x.drop('testcase_ID', axis='columns')

In [8]:
import sklearn

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 0)

# [Neural tutorial](https://towardsdatascience.com/neural-networks-from-scratch-easy-vs-hard-b26ddc2e89c7)

In [10]:
with open('../data/baseline-model-binary','rb') as f:
    model = pickle.load(f)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [11]:
nn_evaluate = model.evaluate(x_test,y_test)

10751/10751 [==============================] - 0s 36us/step


In [12]:
nn_score = nn_evaluate[1]

In [9]:
y_pred = model.predict_classes(x)

In [10]:
import seaborn as sns
from sklearn import metrics
import matplotlib

In [11]:
y_predict = model.predict(x_test)

In [12]:
import numpy as np
from matplotlib.colors import LogNorm

Change Confusion Matrix Below

In [13]:
confusion_matrix = pd.DataFrame(
    data=metrics.confusion_matrix((y_test.values+0).argmax(axis=1), y_predict.argmax(axis=1)), 
)
confusion_figure, confusion_axes = matplotlib.pyplot.subplots()
confusion_figure.set_size_inches(15, 12)
confusion_axes.set_title(
    'Confusion matrix showing the frequency of \n'
    'correct and incorrect bug classification predictions.'
    '\n\n'  # hack to avoid overlap with x-axis labels below
)
sns.heatmap(confusion_matrix, norm=LogNorm(confusion_matrix.min().min(),confusion_matrix.max().max()),
            vmin = 0.001, vmax=1000)


AxisError: axis 1 is out of bounds for array of dimension 1

# Random Forest

In [14]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1, random_state = 42)
# Train the model on training data
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=None,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [16]:
rf.score(x_test, y_test)

-0.016853458189709514

In [17]:
y_pred_rf = pd.DataFrame(rf.predict(x_test))

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
rf_accuracy = accuracy_score(y_test, y_pred_rf)

Sort Visualisation Below!

In [22]:
# from keras.utils import to_categorical

# from visualisation import cve_predict_table

ModuleNotFoundError: No module named 'visualisation'

In [17]:
#example_script = x_test.iloc[0,]
#example_label = y_test.iloc[0,]

In [31]:
#cve_predict_table(data, rf, example_script, example_label)

NameError: name 'keras' is not defined

# KNN

## 1NN

In [70]:
from sklearn.neighbors import KNeighborsClassifier 

oneknn = KNeighborsClassifier(n_neighbors=1,weights = 'distance')
oneknn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='distance')

In [71]:
oneknn_score=oneknn.score(x_test,y_test)

In [73]:
oneknn_score

0.8069946981676123

2NN returns the same as one

## 3NN

In [77]:
from sklearn.neighbors import KNeighborsClassifier 

threeknn = KNeighborsClassifier(n_neighbors=3,weights = 'distance')
threeknn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [78]:
threeknn_score=threeknn.score(x_test,y_test)

In [79]:
threeknn_score

0.7979722816482188

## Gaussian Naive Bayes

In [24]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_gnb = gnb.fit(x_train, y_train).predict(x_test)

In [32]:
total_error = (y_test != y_pred_gnb).sum()

print("Number of mislabeled points out of a total %d points : %d"
       % (x_test.shape[0],total_error))
print("Accuracy " + str(1- (total_error/x_test.shape[0])))

Number of mislabeled points out of a total 10751 points : 4348
Accuracy 0.5955725048832667


In [33]:
gnb_score = 1- (total_error/x_test.shape[0])

## SVM



In [29]:
from sklearn import svm
clf_svm = svm.SVC(kernel = 'poly', gamma='scale')
clf_svm.fit(x_train, y_train)  

y_pred_svm = clf_svm.predict(x_test)

In [34]:
total_error = (y_test != y_pred_svm).sum()

print("Number of mislabeled points out of a total %d points : %d"
       % (x_test.shape[0],total_error))
print("Accuracy " + str(1- (total_error/x_test.shape[0])))

Number of mislabeled points out of a total 10751 points : 1334
Accuracy 0.8759185192075156


In [35]:
svm_score = 1- (total_error/x_test.shape[0])

# Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
y_pred_lr = logreg.predict(x_test)
print('Accuracy ' + str(logreg.score(x_test, y_test)))

Accuracy 0.8430843642451865


In [39]:
logreg_accuracy = logreg.score(x_test, y_test)

# AdaBoost

In [40]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification

In [41]:
clf_ab = AdaBoostClassifier(n_estimators=100, random_state=0)
clf_ab.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

In [42]:
y_pred_ab = clf_ab.predict(x_test)
# y_pred = y_pred.reshape(-1,1)
# y_pred.shape

In [43]:
total_error = (y_test != y_pred_ab).sum()

print("Number of mislabeled points out of a total %d points : %d"
       % (x_test.shape[0],total_error))
print("Accuracy " + str(1- (total_error/x_test.shape[0])))

Number of mislabeled points out of a total 10751 points : 2223
Accuracy 0.7932285368802903


In [44]:
ab_accuracy = 1- (total_error/x_test.shape[0])

# XGBoost

In [53]:
!pip install xgboost

     |████████████████████████████████| 142.8MB 100kB/s eta 0:00:012  |▊                               | 3.4MB 2.6MB/s eta 0:00:54     |█▍                              | 6.1MB 2.6MB/s eta 0:00:53     |█▋                              | 7.4MB 7.0MB/s eta 0:00:20     |█▊                              | 7.7MB 7.0MB/s eta 0:00:20     |██                              | 9.2MB 7.0MB/s eta 0:00:20     |██▏                             | 9.9MB 7.0MB/s eta 0:00:19     |██▋                             | 11.5MB 7.0MB/s eta 0:00:19     |██▊                             | 12.0MB 7.0MB/s eta 0:00:19     |██▉                             | 12.9MB 6.7MB/s eta 0:00:20     |███                             | 13.4MB 6.7MB/s eta 0:00:20     |███▏                            | 14.1MB 6.7MB/s eta 0:00:20     |████▍                           | 19.7MB 6.7MB/s eta 0:00:19     |████▌                           | 20.2MB 6.7MB/s eta 0:00:19     |████▋                           | 20.5MB 6.7MB/s eta 0:00:19     |████▊      

In [55]:
import xgboost as xgb
model_xgb=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model_xgb.fit(x_train, y_train)
model_xgb.score(x_test,y_test)

0.6984466561250117

In [56]:
y_pred_xgb = model_xgb.predict(x_test)

In [57]:
total_error = (y_test != y_pred_xgb).sum()

print("Number of mislabeled points out of a total %d points : %d"
       % (x_test.shape[0],total_error))
print("Accuracy " + str(1- (total_error/x_test.shape[0])))

Number of mislabeled points out of a total 10751 points : 3242
Accuracy 0.6984466561250116


In [58]:
xgb_accuracy = 1- (total_error/x_test.shape[0])

# Table of Results

In [59]:
results = {'Model': ['Neural Network', 'Random Forest', '1 Nearest Neighbour', '3 Nearest Neighbours', 'Gaussian Naive Bayes', 'SVM', 'Logistic Regression', 'AdaBoost', 'XGBoost'], 'Accuracy' : [nn_score, rf_accuracy, oneknn_score, threeknn_score, gnb_score, svm_score, logreg_accuracy, ab_accuracy, xgb_accuracy]}
accuracy_table = pd.DataFrame(data = results)

In [60]:
accuracy_table.sort_values(by = ['Accuracy'], ascending=False)

,Model,Accuracy
0,Neural Network,0.912845
5,SVM,0.875919
6,Logistic Regression,0.843084
2,1 Nearest Neighbour,0.806995
3,3 Nearest Neighbours,0.797972
7,AdaBoost,0.793229
1,Random Forest,0.745791
8,XGBoost,0.698447
4,Gaussian Naive Bayes,0.595573


# Stacking

In [62]:
!pip install vecstack

  Stored in directory: /root/.cache/pip/wheels/35/6d/ca/bce17942bcf7c267b13c97c9c95e2f0ecf0b42160e6074f448
Successfully built vecstack


In [63]:
from vecstack import stacking

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [80]:
models = [clf_svm, logreg, oneknn]

In [81]:
S_train, S_test = stacking(models,                   
                           x_train, y_train, x_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [SVC]
    fold  0:  [0.86649312]
    fold  1:  [0.86607143]
    fold  2:  [0.87272051]
    fold  3:  [0.85746185]
    ----
    MEAN:     [0.86568673] + [0.00542957]
    FULL:     [0.86568691]

model  1:     [LogisticRegression]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  0:  [0.85050205]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  1:  [0.83928571]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  2:  [0.82471157]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


    fold  3:  [0.82954968]
    ----
    MEAN:     [0.83601225] + [0.00987598]
    FULL:     [0.83601525]

model  2:     [KNeighborsClassifier]
    fold  0:  [0.79471923]
    fold  1:  [0.79055060]
    fold  2:  [0.79233346]
    fold  3:  [0.80089319]
    ----
    MEAN:     [0.79462412] + [0.00390996]
    FULL:     [0.79462376]



In [86]:
level_two = clf_svm.fit(S_train, y_train)
y_pred_stack = level_two.predict(S_test)

In [87]:
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred_stack))

Final prediction score: [0.89154497]
